In [ ]:
import pandas as pd

In [ ]:
#Don't use for testing, too long to download data everytime
#df = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv')
#df.head()

In [ ]:
init_df = pd.read_csv('../Data/CovidData/owid-covid-data.csv')
init_df

In [ ]:
df = init_df.copy()
df

In [ ]:
df.continent.unique()

In [ ]:
nan_df = df.loc[df['continent'].isna()]
nan_df.head()


In [ ]:
df.date.unique() 

In [ ]:
eur_df = df.loc[df['continent']=='Europe']

In [ ]:
eur_df.location.unique()

In [ ]:
print(eur_df.total_cases.isna().any())
print(eur_df.total_deaths.isna().any())

In [ ]:
eur_df['total_cases'].fillna(0, inplace=True)
eur_df['total_deaths'].fillna(0, inplace=True)

In [ ]:
print(eur_df.new_cases.isna().any())
print(eur_df.new_deaths.isna().any())

In [ ]:
def split_dfs(df):
    eur_df = df.loc[df['continent'] == 'Europe']
    s_ame_df = df.loc[df['continent'] == 'South America']
    n_ame_df = df.loc[df['continent'] == 'North America']
    asia_df = df.loc[df['continent'] == 'Asia']
    afr_df = df.loc[df['continent'] == 'Africa']
    ocea_df = df.loc[df['continent'] == 'Oceania']
    
    return {'Europe':eur_df,'North America':n_ame_df, 'South America':s_ame_df, 'Asia':asia_df,'Oceania':ocea_df, 'Africa':afr_df}


def preprocessing_continent(df):
    df.dropna(axis=0, subset=['continent'], inplace=True)
    
    df['total_cases'].fillna(0, inplace=True)
    df['total_deaths'].fillna(0, inplace=True)
    df['new_cases'].fillna(0, inplace=True)
    df['new_deaths'].fillna(0, inplace=True)
    
    #Some countries have negative new_cases, these will just be inverted
    df['new_cases'] = df['new_cases'].abs()
    df['new_deaths'] = df['new_deaths'].abs()
    df['new_cases_smoothed'] = df['new_cases_smoothed'].abs()
    df['new_deaths_smoothed'] = df['new_deaths_smoothed'].abs()
    


def check_na(df):
    tot_cases = df['total_cases'].isna().any()
    tot_deaths = df['total_deaths'].isna().any()
    new_cases = df['new_cases'].isna().any()
    new_deaths = df['new_deaths'].isna().any()
    
    output = 'Has NA:' , '\nTotal_cases: ' ,tot_cases , '\nTotalDeaths ' , tot_deaths , '\nNew_cases: ' , new_cases , '\nNew_deaths ' , new_deaths 
    return(output)


In [ ]:
print(check_na(eur_df))

In [ ]:
preprocessing_continent(eur_df)

In [ ]:
eur_df['new_cases'] = eur_df['new_cases'].abs()

In [ ]:
px.line(eur_df.loc[eur_df['location']=='United Kingdom'], 'date', ['new_cases', 'new_cases_smoothed'])

In [ ]:
px.line(eur_df.loc[eur_df['location']=='France'], 'date', 'new_cases_smoothed', color='location')


In [ ]:
#Get continent dfs
eur_df, s_ame_df, n_ame_df, asia_df, afr_df, ocea_df = split_dfs(df)
print(eur_df.location.unique())
print(s_ame_df.location.unique())
print(n_ame_df.location.unique())
print(asia_df.location.unique())
print(afr_df.location.unique())
print(ocea_df.location.unique())

In [ ]:
preprocessing_continent(ocea_df)

In [ ]:
fig = px.choropleth(useDf, geojson=gj, locations='regionId', color='regionCases',
                          color_continuous_scale='Reds', featureidkey='properties.name', projection='mercator',
                animation_frame='lastUpdatedAtApify')
fig.update_geos(showland=True, landcolor="Green",
                showocean=True, oceancolor="LightBlue",
                showlakes=True, lakecolor="LightBlue",
                showcountries=True, countrycolor="Black", resolution=50,
                scope="world", showsubunits=True, subunitcolor="White",
                fitbounds='locations', visible=False)
fig.update_layout(margin={'r':0,'t':0,'l':0,'b':0})
fig.show()

In [ ]:
fig = px.choropleth(ocea_df, locations='iso_code',
                   color='new_cases')
#, center=dict(lat=22.7359, lon=140.0188)
fig.update_geos( projection_scale=1)

fig.update_layout(margin=dict(r=0, t=0, l=0, b=0))
fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Choropleth(
    locations=eur_df['iso_code'],
    z = eur_df['new_cases']))


#fig.update_geos(center_lon=15.2551, center_lat=54.5260, lataxis_range=[0,55], lonaxis_range=[0,100])
fig.update_geos(scope='world')
fig.update_layout(margin=dict(r=0,t=0,l=0,b=0))

fig.show()

In [ ]:
df = init_df.copy()
preprocessing_continent(df)
dfs = split_dfs(df)
fig = go.Figure()

buttons=[]
region_plot_names=[]
default_continent = next(iter(dfs))


for key, df in dfs.items():
    fig.add_trace(go.Choropleth(locations=df['iso_code'], z=df['new_cases'], visible=(key==default_continent)))
    region_plot_names.extend([key])
    print(region_plot_names)

for key, df in dfs.items():
    #If broken change method to 'update'
    buttons.append(dict(method='restyle', label=key, args=[{'visible':[key==k for k in region_plot_names]}]))

fig.update_layout(
    updatemenus=[{'buttons': buttons, "direction": 'down', 'active':0,'showactive':True}])

fig.show()

In [ ]:
df = init_df.copy()
preprocessing_continent(df)
dfs = split_dfs(df)


In [ ]:
df = init_df.copy()
preprocessing_continent(df)
dfs = split_dfs(df)
eur_df = dfs.get('Europe').copy()
eur_df

In [ ]:
#Clean df
import math
df = init_df.copy()
df = df.loc[df['date']=='2020-07-09']
preprocessing_continent(df)
dfs = split_dfs(df)

#Bubble chart
import plotly.graph_objects as go
fig = go.Figure()


#Log to the base 2, or 10

for key, df in dfs.items():
    #Lambda with ternary: checks if the country has new cases, if it does divide the new cases by 10 else bubble size is 1
    fig.add_trace(go.Scatter(x=df['gdp_per_capita'], y=df['life_expectancy'], name=key, marker_size=df.apply(lambda d : (math.log(d['new_cases'], 2)) if d['new_cases'] != 0 else 1 , axis=1), text=("Country: " + df['location'] + "\nCases: ")))


fig.update_traces(mode='markers', marker=dict(sizemode='area'))
fig.show()


##David: As a user i want to be able to find the UK how can I do that?
##Allow user to change axis
##Log to the base 2, or 10

In [ ]:
main_df = init_df.copy()
preprocessing_continent(main_df)
main_df.continent.unique()

In [ ]:
import plotly.graph_objects as go
import numpy as np
import math

#Df initialise
main_df = init_df.copy()
preprocessing_continent(main_df)
dfs = split_dfs(main_df)
DATES = np.sort(main_df.date.unique())
CONTINENTS = np.sort(main_df.continent.unique())

fig_dict=dict(
    data=list(),
    layout=dict(),
    frames=list()
)


#For 
fig_dict["layout"]["updatemenus"] = [dict(
    buttons=[dict(
        method="animate",
        label="Play",
        args=[None, dict(
            frame=dict(
                duration=500, 
                redraw=False
            ), 
            fromcurrent=True)]
        
    ), dict(
        method="animate",
        label="Pause",
        args=[[None], dict(
            frame=dict(
                duration=0,
                redraw=False
            ),
            mode="immediate"
        )]
    )],
    direction="left",
    showactive=False,
    type="buttons",
    
    
)]

#Buttons
fig_dict["layout"]

date=DATES[0]
for continent,df in dfs.items():
    df.loc[df['date']==date]
    trace_args = dict(
        x=df['gdp_per_capita'],
        y=df['life_expectancy'],
        name=continent,
        mode="markers",
        marker_size=df.apply((lambda df: math.log(df['new_cases'],2) if df['new_cases']!=0 else 1), axis=1),
        marker= dict(
            sizemode="area")
        )
    fig_dict["data"].append(trace_args)
    
#Frames
for date in DATES[1:20]:
    print("ENTERING LONG ASS LOOP")
    frame = dict(
        data=list(),
        name=date
    )
    for continent,df in dfs.items():
        df.loc[df['date']==date]
        #Set the trace parameters for current continent date
        trace_args = dict(
            x=df['gdp_per_capita'],
            y=df['life_expectancy'],
            name=continent,
            mode="markers",
            marker_size=df.apply((lambda df: math.log(df['new_cases'],2) if df['new_cases']!=0 else 1), axis=1),
            marker= dict(
                sizemode="area")
            )
        frame['data'].append(trace_args)
    #End for continent
    fig_dict['frames'].append(frame)
    #Make the slider
    slider_step = dict()
#End for day
    

fig = go.Figure(fig_dict)
fig.show()

In [ ]:
fig_dict= dict(
"data" = list())

In [ ]:
##Change the values of date to weekly instead of by day